In [1]:
import math

import numpy
import pandas

import ipywidgets as widgets
from IPython.display import display, clear_output

import gensim
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import pythainlp

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans

from data_tokenizer import load_corpus

from model.new_sdc import NewSDC
from model.sdc import SDC

### Load Data

In [2]:
file_name = 'Contrast - ที่นั่งคนอ้วน'

corpus = load_corpus('../data/' + file_name + '.txt')

print('Total documents', len(corpus))

f = open('../data/tokenized/tokenized_' + file_name + '.txt')
tokenized_corpus = eval(f.read())
f.close()

Total documents 319


### Preprocess Corpus

#### Remove Words

In [3]:
dictionary = Dictionary(tokenized_corpus)
print('origin:', len(dictionary), 'words')

dictionary.filter_extremes(no_below=2, no_above=0.7, keep_n=len(dictionary))
print('filter frequent words:', len(dictionary), 'words')

letter_words = [id for id in range(len(dictionary)) if len(dictionary[id]) <= 1] 
dictionary.filter_tokens(bad_ids=letter_words)
print('filter letter words:', len(dictionary), 'words')

stopwords = pythainlp.corpus.stopwords.words('thai')
stopwords.extend(['นี้'])
dictionary.add_documents([stopwords])
stopwords = [dictionary.token2id[word] for word in stopwords]
dictionary.filter_tokens(bad_ids=stopwords)
print('filter stop words:', len(dictionary), 'words')

origin: 1363 words
filter frequent words: 572 words
filter letter words: 568 words
filter stop words: 367 words


In [4]:
idx_corpus = [dictionary.doc2idx(doc) for doc in tokenized_corpus]

temp_corpus = []
for doc in idx_corpus:
    temp_corpus.append([dictionary[id] for id in doc if id >= 0])
idx_corpus = temp_corpus

#### Dimension Reduction

In [5]:
average_doc_size = 0
for doc in idx_corpus:
    average_doc_size += len(doc)
average_doc_size /= len(idx_corpus)
average_doc_size = math.ceil(average_doc_size)

df = dictionary.dfs
filtered_corpus = []
for doc in idx_corpus:
    new_doc = [(word, df[dictionary.token2id[word]]) for word in doc]
    new_doc.sort(reverse=True, key=lambda x: x[1])
    new_doc = new_doc[:average_doc_size]
    filtered_corpus.append([word for word, df in new_doc])

### Clustering

In [6]:
def get_onehot(corpus, weight):
    dictionary = Dictionary(corpus)
#     dictionary.filter_extremes(no_below=2, no_above=1, keep_n=len(dictionary))

    bow_corpus = [dictionary.doc2bow(doc) for doc in corpus]
    if weight == 'normal':
        weight_corpus = bow_corpus
    elif weight == 'tfidf':
        tfidf = TfidfModel(bow_corpus, smartirs='ltc')
        weight_corpus = [tfidf[doc] for doc in bow_corpus]

    unique_words = [dictionary[id] for id in range(len(dictionary))]
    array = numpy.zeros((len(corpus), len(unique_words)), dtype=float)
    for i, doc in enumerate(weight_corpus):
        for id, score in doc:
            array[i, id] = score

        if weight == 'normal' and len(doc) != 0:
#             array[i] = numpy.divide(array[i], len(idx_corpus[i]))
            array[i] = numpy.divide(array[i], len(doc))
    
    return pandas.DataFrame(array, columns=unique_words, dtype=float)

In [7]:
def generate_result(predicted_labels, marks):
    result = pandas.DataFrame()
    result['comment'] = corpus
    result['tokenized_comment'] = idx_corpus
    result['predicted_label'] = predicted_labels
    if marks:
        result['marks'] = marks
    else:
        result['marks'] = -1
    return result

In [25]:
def eval_cluster(onehot_corpus, result):
    num_cluster = len(numpy.unique(result['predicted_label']))

    clusters = [[] for i in range(num_cluster)]
    corpus_centroid = []
    for i, label in result['predicted_label'].iteritems():
        clusters[label].append(numpy.array(onehot_corpus.iloc[i]))
        corpus_centroid.append(numpy.array(onehot_corpus.iloc[i]))
    corpus_centroid = numpy.mean(corpus_centroid, axis=0).reshape(1, -1)   

#     print('\tIntra cluster sim\tInter cluster sim\tIntra / Inter')
    compactness = 0
    centroids = []
    for i in range(num_cluster):
        size = len(clusters[i])
        if size != 0:
            centroid = numpy.mean(clusters[i], axis=0)
            centroids.append(centroid)
            centroid = centroid.reshape(1, -1)
            similarities = cosine_similarity(centroid, clusters[i])
            compactness += numpy.sum(similarities)

#             intra = numpy.sum(similarities) / size
#             inter = cosine_similarity(centroid, corpus_centroid)[0][0]
#             print(i, end='\t')
#             print(intra, end='\t')
#             print(inter, end='\t')
#             print(intra / inter)
    return compactness, centroids

In [9]:
min_samples = 7
eps = 0.32
expand_rate = 0.05
epoch = 15

onehot_corpus = get_onehot(idx_corpus, 'normal')

In [ ]:
max_compactness = 0
for i in range(epoch):
    model = NewSDC()
    _tpredicted_labels, marks = model.predict(onehot_corpus, min_samples, eps, expand_rate)

#     model = SDC()
#     _tpredicted_labels, marks = model.predict(onehot_corpus, min_samples, eps, expand_rate)
    
#     marks = None
    
#     model = DBSCAN(metric='cosine', eps=eps, min_samples=min_samples).fit(onehot_corpus)
#     _tpredicted_labels = model.labels_ + 1

#     model = KMeans(n_clusters=14).fit(onehot_corpus)
#     _tpredicted_labels = model.labels_
    
    _tresult = generate_result(_tpredicted_labels, marks)
    compactness, _tcentroids = eval_cluster(onehot_corpus, _tresult)
    
    if compactness > max_compactness:
        max_compactness = compactness
        predicted_labels = _tpredicted_labels
        result = _tresult
        centroids = _tcentroids
        
print(max_compactness)
label_count = numpy.unique(result['predicted_label'], return_counts=True)[1]
num_cluster = len(label_count)
print(label_count)

#### Iterative New SDC

In [10]:
centroids = None 
prev_label_count = None
while True:
    model = NewSDC()
    predicted_labels, marks = model.predict(onehot_corpus, min_samples, eps, expand_rate, seeds=centroids)
    
    result = generate_result(predicted_labels, marks)
    compactness, centroids = eval_cluster(onehot_corpus, result)
    
    label_count = numpy.unique(result['predicted_label'], return_counts=True)[1]
    if numpy.array_equal(label_count, prev_label_count):
        break
    prev_label_count = label_count
    centroids = centroids[1:]
    
    print(compactness)
    print(label_count)
num_cluster = len(label_count)

158.99703547097505
[67 79 20 14 30  8 10 25 18  8 10  7  8  8  7]
146.55864913906223
[ 83 121  31  12  29  10  11  15   7]
146.0207524780847
[ 79 128  31  12  26  10  11  15   7]
144.76959222402547
[ 84 126  31  11  24  10  11  15   7]
144.42108623333021
[ 84 127  31  11  23  10  11  15   7]


#### Grouping

In [11]:
sims = cosine_similarity(centroids)
new_labels = [i for i in range(num_cluster)]
print(new_labels)
for i, row in reversed(list(enumerate(sims))):
    for j, value in reversed(list(enumerate(row[:i + 1]))):
        if i != j and value >= eps - eps / 20:
            print(i, j, value)
            base = min(new_labels[i], new_labels[j])
            new_labels[j] = base
            new_labels = [base if label == new_labels[i] else label for label in new_labels]
print(new_labels)

grouped_labels = numpy.zeros(len(corpus))
for i, label in enumerate(predicted_labels):
    grouped_labels[i] = new_labels[label]
new_result = generate_result(grouped_labels, None)

[0, 1, 2, 3, 4, 5, 6, 7, 8]
4 1 0.488899447292611
[0, 1, 2, 3, 1, 5, 6, 7, 8]


### Result

In [12]:
class Widget:
    def __init__(self, result, column_name):
        self.result = result
        self.column_name = column_name
        
        label_count = numpy.unique(result['predicted_label'])
        self.widget = widgets.ToggleButtons(
            options=[int(num) for num in label_count],
            disabled=False,
            button_style='',
        )
        
        self.widget.observe(self.on_click, names='index')
        self.on_click({'new' : 0})
        
    def on_click(self, change):
        clear_output()
        display(self.widget)
        new = self.widget.options[change['new']]
        for index, value in self.result[self.result['predicted_label'] == new].iterrows():
            if value['marks'] == 0:
                print("@", end="")
            elif value['marks'] == 1:
                print("*", end="")
            print(index, value[self.column_name])

In [14]:
# result.to_csv('../data/results/filtered/' + file_name + '.csv', index=False)

# result = pandas.read_csv('../data/results/filtered/' + file_name + '.csv')

print(eval_cluster(onehot_corpus, result)[0])
print(numpy.unique(result['predicted_label'], return_counts=True)[1])

144.42108623333021
[ 84 127  31  11  23  10  11  15   7]


In [15]:
w1 = Widget(new_result, 'comment')

ToggleButtons(options=(0, 1, 2, 3, 5, 6, 7, 8), value=0)

7 อีกหนึ่งเหตุผลที่กุต้องไปออกกำลังกาย
16 เกือบละ D
17 เคยมีฝรั่งฟ้องสายการบินได้หลายล้านเลย
18 ถ้าฟ้องรถตู้อาจจะได้ 50.-
28 😿 เพิ่งเจอในลิฟท์เย็นนี้นี่เอง .. สองนาง Big Size เสียงดังโฉ่งฉ่าง เข้าใจว่ากำลังคุยกับเพื่อนคงอรรถรส แต่เข้ามาเบียดเราที่ยืนอยู่ก่อนกระเด็นฟาดผนังลิฟท์ไปเลยจ้าา คิดว่าเค้าจะขอโทษ..ก็ไม่ 😂 ... ได้แต่คิดว่าเค้าคงไม่รู้ตัวคงไม่ได้ตั้งใจ 🤕
32 "ทำไมไม่ไห้คนตัวใหญ่เค้ากินเนื้อที่จ่ายแพงกว่า"เค้าอาจจะชอบโคขุนราชรีมากกว่าเนื้อวัวโกเบก็ได้ ทำไมต้องบังคับไห้เค้ากินเนื้อที่เค้าไม่ชอบอ่ะ อีกอย่างถ้าตัวใหญ่นั่งกินเนื้อมันกว่าจะอิ่มมันก็แพงอยู่แล้วป่ะ ยังจะไห้เค้ากินเนื้อเกรดสูงๆอีก
42 ใช่ใช่ไหม ???
46 อ่านโพสนี้แล้ว บอกกับตัวเอง "กูต้องลดน้ำหนักโว้ยยย กูต้องทำให้ได้"ขอโทษ ถ้าครั้งนึงเคยไปเบียดเบียนพื้นที่สาธารณะของใครหลายๆคน ต่อไปนี้เราจะพยายามลดน้ำหนักแล้วนะ ขอบคุณที่เตือนกัน 💪🏻💪🏻💪🏻💪🏻
54 อยากให้พระเจ้าเสกให้โลกนี้มีแต่คนผอมหน้าตาพิมเดียวกันหมด หุ่นเท่ากันหมด พื้นฐาน ฐานะ ชีวิตเท่ากันหมดทั้งโลกจะได้ไม่ต้องมีเรื่องเหยียดกัน
60 เรามีตัวอย่าง 2 เหตุการณ์จะเล่าให้ฟังเรื่องแรก เพ

In [16]:
w2 = Widget(result, 'comment')

ToggleButtons(index=1, options=(0, 1, 2, 3, 4, 5, 6, 7, 8), value=1)

*0 จขกท. คุยเรื่องนี้กับแอดนะครับ เค้ารู้ว่าคนอ้วนหรือคนตัวใหญ่นั้นไม่ผิด แต่ถ้าจะโทษ เค้าโทษที่ระบบการจัดการการออกมาตราการขนส่งนะครับ มันเป็นหน้าที่การจัดการอย่างนึงนะครับ เรื่องจัดเตรียมที่นั่ง หรือการจ่ายตามอัตราพื้นที่ที่นั่ง.เราส่งพัสดุยังมีเรทราคาตามน้ำหนักเพราะอะไร นั่นแหละครับมันคือเรทราคาที่คิดตามพื้นที่ใช้สอยในการขนส่งแต่ละครั้ง.ถามว่าดูเป็นการเอาแต่ใจมั้ย?มาดูจริงๆก็ไม่นะ แค่เรียกร้องสิทธิ์ที่ควรได้ .คือไม่ได้ว่า จ่าย50.- แล้วต้องได้บริการ 500.- คือจ่าย50.- ก็ต้องการบริการ50.- นั่นแหละ .ทีนี้การนั่งแบบนี้ เค้าก็อึดอัด ทั่งที่ควรได้นั่งสบายเหมือนคนทั่วๆไปที่จ่าย50.- เช่นกัน .บางคนบอกทำไมไม่ไปนั่งแท๊กซี่บลาๆ มันก็แก้ได้นะแต่มันดูเป็นการแก้ที่ปลายเหตุ อีกทั้งยังดูเหมือนผลักภาระมาให้อีกทั้งที่มันไม่ใช่ความผิดเค้าเลย (คนตัวใหญ่หรืออ้วนก็ไม่ผิด)มันไม่ใช่แค่ จขกท. ที่อึดอัด คนตัวใหญ่หรือคนอ้วนเองก็อึดอัดเหมือนกัน เค้าก็อยากให้เกิดการเปลี่ยนแปลงนั่นแหละ วันนี้ยังไม่เปลี่ยน 5ปี10ปี ก็ไม่แน่อันนี้เข้าใจทุกฝ่ายนะ คนนึงก็อึดอัด อีกคนนึงก็รู้สึกไม่ดีถือว่าโพสต์นี้ทำให้ได้เห็นมุมมองหลายๆค

In [16]:
w3 = Widget(result, 'tokenized_comment')

ToggleButtons(index=8, options=(0, 1, 2, 3, 4, 5, 6, 7, 8), value=8)

*18 ['รัฐบาล', 'ตอน', 'ปัญหา', 'ปัญหา', 'อากาศ', 'ประชาชน', 'ประเทศ', 'แก้', 'ปี', 'หายใจ', 'สุขภาพ', 'ขนาด', 'เค้า', 'สนใจ', 'จุด']
*55 ['หมอ', 'รัฐบาล', 'ตัว', 'รู้', 'รู้', 'สัก', 'ประกาศ', 'ป้องกัน', 'ขอบคุณ', 'อันตราย', 'ตื่นตัว', 'รูปธรรม']
*84 ['รัฐบาล', 'ตัว', 'รัฐบาล', 'ตอน', 'อากาศ', 'ประชาชน', 'ปี', 'ขนาด', 'ป้องกัน', 'พิษ', 'พิษ', 'ปล่อย', 'นิ่ง', 'วิกฤติ', 'ล่ะ']
*147 ['รัฐบาล', 'สัก', 'ข้อ']
*160 ['รัฐบาล', 'รัฐบาล', 'ดี', 'รอ', 'ประเทศไทย', 'บ้านเมือง', 'ปชช']
*164 ['เรื่อง', 'รัฐบาล', 'ปัญหา', 'แก้', 'ขนาด', 'มัว', 'วิธี', 'ผลิต']
*183 ['รัฐบาล']
*190 ['รัฐบาล', 'ถาม', 'เวลา']


In [19]:
seed = 0
compare = 0

a = numpy.array(onehot_corpus.iloc[seed]).reshape(1, -1)
b = numpy.array(onehot_corpus.iloc[compare]).reshape(1, -1)
print(cosine_similarity(a,b))

print(idx_corpus[seed])
print(corpus[seed])
print(idx_corpus[compare])
print(corpus[compare])

[[1.]]
['ประกาศ', 'หยุด', 'เรียน', 'ระดับ', 'ชั้น', 'บริหาร', 'โรง', 'เรียน', 'ประกาศ', 'ชม']
รร.รุ่งอรุณประกาศหยุดเรียนทุกระดับชั้น ชื่นชมผู้บริหารโรงเรียนในการตัดสิ้นใจค่ะ ประกาศเมื่อราว 1 ชม.ที่ผ่านมาค่ะ
['ประกาศ', 'หยุด', 'เรียน', 'ระดับ', 'ชั้น', 'บริหาร', 'โรง', 'เรียน', 'ประกาศ', 'ชม']
รร.รุ่งอรุณประกาศหยุดเรียนทุกระดับชั้น ชื่นชมผู้บริหารโรงเรียนในการตัดสิ้นใจค่ะ ประกาศเมื่อราว 1 ชม.ที่ผ่านมาค่ะ
